# Part 3: From Viewshed to Viewscape 

In the third part, we will demonstrate the use of GRASS for a small viewshed case study.
The goal is to **analyze the area a driver would see from a road**.
This notebook can be run only after notebook Part 2 was executed.

Topics covered:
 * raster algebra ([r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html))
 * raster mask ([r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html))
 * raster as a numpy array ([grass.script.array](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#module-script.array))
 * statistics, data wrangling, and plotting with GRASS directly and together with pandas and seaborn

Load Python libraries. (This may be be skipped in the Google Colab notebook where we already did this  at the beginning of the single, long notebook.)

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS packages we need.
import grass.script as gs
import grass.jupyter as gj

In the previous notebook (Part 1) we created new project *dix_park*. This automatically created new default mapset (subproject) _PERMANENT_ where we then imported our base data. Now it's time to create a new mapset for our viewshed analysis, we will name it _viewshed_:

In [ ]:
from grass.experimental import require_create_ensure_mapset

# Create mapset if it does not exist and continue if it already exists.
# (By deafult, the function requires the mapset to exist.)
require_create_ensure_mapset("dix_park/analysis", ensure=True)

Start a new session in the mapset:

In [ ]:
# Start GRASS session
session = gj.init("dix_park/analysis")

## Organizing and Accessing Data Across Mapsets

A project consists of one or more mapsets which facilitate data organization and protection. Mapsets act as subprojects or namespaces.

Using _g.mapsets_ with the _l_ flag, we can see that our current project has now these three mapsets:

In [ ]:
gs.run_command("g.mapsets", flags="l")

By default, we can see data in our current mapset and in the default mapset called PERMANENT. We can see that by using _g.mapsets_ with the _l_ flag:

In [ ]:
gs.run_command("g.mapsets", flags="p")

While we see the data in PERMANENT and they are available for reading, creating or editing data is only possible in the current mapset, protecting the data from unintentional modifications.

The _g.list_ tools can list the data we currently see:

In [ ]:
gs.run_command("g.list", type="raster")

As the list of data suggests, the data can be accessed without specifying the mapset. Using _r.info_, we show metadata of a raster in the PERMANENT mapset:

In [ ]:
gs.run_command("r.info", map="dsm")

When needed, mapset name can be used explicitly with name-at-mapset syntax written like `name@mapset`. This is also know as the _full name_ or _fully qualified name_ within a project. Full names are useful when we have the same or similar names for data in multiple mapsets. In our case, we don't have to use mapset name explicilty to make that distinction, but we can:

In [ ]:
gs.run_command("r.info", map="dsm@PERMANENT")

We can list the data using the full names by adding the _m_ flag to the _g.list_ call:

In [ ]:
gs.run_command("g.list", type="raster", flags="m")

Using the full name, we can access data in any mapset (within the project). Again, this data is accessible for reading, so we can, for example, get metadata of a raster we created before:

In [ ]:
gs.run_command("r.info", map="viewshed_10@viewshed")

We can avoid using the full name for data in specific mapset or mapsets by adding the mapset to the list called _search path_ which is a list of mapsets where data is searched for when the name is used without a mapset:

In [ ]:
gs.run_command("g.mapsets", mapset="viewshed", operation="add")

Now the _g.mapsets_ tool with _p_ flag tells us the three, not two, mapsets which are on our search path:

In [ ]:
gs.run_command("g.mapsets", flags="p")

When we list data, we see data from the there mapsets on our search path:

In [ ]:
gs.run_command("g.list", type="raster")

## Cumulative viewshed
We can compute the cumulative viewshed, which aggregates viewsheds from multiple viewpoints. In this way you can e.g., identify the most frequently visible areas from the road.


Before the computation, we set the computational region to the raster we used to compute the viewsheds.

In [ ]:
gs.run_command("g.region", region="umstead_drive_region")

In [ ]:
%%bash
g.list pattern="viewshed_[0-9]+" type="raster" -e 

In [ ]:
maps = gs.list_strings(type="raster", pattern="viewshed_[0-9]+", flag="e")

Since our viewshed rasters are binary (0 invisible, 1 visible), we will use r.series method *sum*. Then we replace zeros with no data using r.null and set a new color ramp:

In [ ]:
# cumulative viewshed
gs.run_command("r.series", input=maps, output="cumulative_viewshed", method="sum")
gs.run_command("r.null", map="cumulative_viewshed", setnull=0)
gs.run_command("r.colors", map="cumulative_viewshed", color="plasma")

Let's visualize the results:

In [ ]:
cumulative_map = gj.InteractiveMap()
cumulative_map.add_raster("cumulative_viewshed", opacity=0.8)
cumulative_map.add_vector("umstead_drive")
cumulative_map.add_layer_control(position="bottomright")
cumulative_map.show()

And create a 3D rendering with draped cumulative viewshed over the DSM (this won't work in Google Colab):

In [ ]:
map3d = gj.Map3D()
map3d.render(
    elevation_map="dsm",
    resolution_fine=1,
    color_map="cumulative_viewshed",
    vline="umstead_drive",
    vline_width=3,
    vline_color="white",
    light_brightness=50,
    position=[0.4, 0.8],
    height=3000,
    perspective=10,
)
map3d.overlay.d_legend(
    raster="cumulative_viewshed", at=(0, 30, 1, 7), use=[1, 2, 3, 4, 5, 6], flags="fb"
)
map3d.show()

## Statistics of Continuous Variable
Now let's compute Excess Green Index (ExGI, [Woebbecke et al.](https://elibrary.asabe.org/abstract.asp?aid=27838)) from orthophoto and analyze its distribution within the visible area.

Break down the RGB image into RGB components:

In [ ]:
gs.run_command("r.rgb", input="ortho", red="red", green="green", blue="blue")

Compute ExGI using raster algebra:

In [ ]:
gs.mapcalc("exgi = (2.0 * green - red - blue)")

In [ ]:
green_map = gj.Map()
green_map.d_rast(map="exgi")
green_map.d_legend(raster="exgi", flags="t")
green_map.show()

In [ ]:
gs.run_command("r.relief", input="ground", output="relief")

green_map = gj.Map()
green_map.d_shade(shade="relief", color="exgi", brighten=50)
green_map.d_legend(raster="exgi", flags="t")
green_map.show()

We will use mask, to limit the analysis only to the data in the visible area:

In [ ]:
gs.run_command("r.mask", raster="cumulative_viewshed")
data = gs.parse_command("r.univar", map="exgi", flags="g")
print(
    f"Average ExGI of visible cells: {float(data['mean']):.2f} ± {float(data['stddev']):.2f}"
)

In [ ]:
green_map = gj.Map()
green_map.d_rast(map="exgi")
green_map.show()

## Histogram

Let's see the histogram of visible ExGI using d.histogram (we are still using the mask):

In [ ]:
histo = gj.Map(width=800, height=400)
histo.d_histogram(map="exgi", nsteps=50)
histo.show()

The low and missing bars are suspitious as we expepect the values to be continuous. However, when we look at the report about values in one of the underlying rasters, using _r.report_, we see that it is not continous either. This is given by spectral resolution of the original data.

In [ ]:
gs.run_command("r.report", map="green")

It is also easy to use the results as a numpy array and then use other Python libraries to analyze the data:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import grass.script.array

In [ ]:
exgi = gs.array.array(mapname="exgi", null="nan")
exgi

In [ ]:
sns.set_style("darkgrid")
sns.histplot(exgi.ravel(), kde=True)
plt.show()

Finally, remove the mask:

In [ ]:
gs.run_command("r.mask", flags="r")

## Categorical Data

For our excercise, we will create categorical data from the ExGI raster by a simple threshold with raster algebra:

In [ ]:
gs.run_command("r.mapcalc", expression="vegetated = if(exgi > 20, 1, 0)")

To label the categories, we will use _r.category_ and will provide it with labeling rules as standard text input (_stdin_), using the _write_command_ function with parameter _rules_ set to `"-"`.

In [ ]:
rules = """
1:vegetated
0:not vegetated
"""
gs.write_command(
    "r.category", map="vegetated", separator=":", rules="-", stdin=rules
)

We will also assign colors to each category in a similar way:

In [ ]:
rules = """
1 #9ed528
0 #61709c
"""
gs.write_command(
    "r.colors", map="vegetated", rules="-", stdin=rules
)

While we are not using DSM for our classification of areas which are vegetated and not vegetated, let's use DSM for the relief background as a check.

In [ ]:
gs.run_command("r.relief", input="dsm", output="dsm_relief")

Using the DSM-based relief and Umstead Drive for context, show the vegetation map:

In [ ]:
green_map = gj.Map()
green_map.d_shade(shade="dsm_relief", color="vegetated", brighten=50)
green_map.d_vect(map="umstead_drive", width=3, color="white")
green_map.d_legend(raster="vegetated", flags="fbc", at=(60,80,70,75), title="ExGI-based Vegetation")
green_map.show()

Most of the trees are identified as not vegetated. That's unexpected even for our naive threshold approach. However, when we look at the orthophoto which is actually our source data, it is clear that most of the trees appear brown on this leaf-off orthophoto.

In [ ]:
green_map = gj.Map(use_region=True)
green_map.d_rast(map="ortho")
green_map.d_vect(map="umstead_drive", width=3, color="white")
green_map.show()

With information about vegetation presence ready, let's get the specific number of cells in each of our categories (classes). (We are using number of cells, but we could also use area instead.) We will use _r.stats_ to give us these counts:

In [ ]:
data = gs.read_command("r.stats", input="vegetated", flags="cnl", separator=",")
print(data)

This statistics is for our whole study area defined by the computational region. To do our actual analysis, we want to limit that to a viewshed, using _r.mask_. We will start by limiting the analysis to the cumulative viewshed we created. To get the data from _r.stats_ into Python, we could string basic manipulation such as `line.split(",")`, but here we will use _csv.DictReader_ from standard Python library because that's more robust and readable. We don't do anything with the data yet, we just print them.

In [ ]:
gs.run_command("r.mask", raster="cumulative_viewshed")
data = gs.read_command("r.stats", input="vegetated", flags="cnl", separator=",")

import csv
import io

reader = csv.DictReader(
    io.StringIO(data), delimiter=",", fieldnames=["category", "label", "count"]
)
for row in reader:
    print(row)

gs.run_command("r.mask", flags="r")

Now, let's collect the number of vegetated and non vegetated cells for each of the viewsheds. This adds another for loop. We will use _r.mask_ to limit the counting done by _r.stats_ just to one viewshed. We will create a new table as a list where each row is a dictionary with viewshed name and counts of cells of each kind.

In [ ]:
result = []
for viewshed in maps:
    gs.run_command("r.mask", raster=viewshed, maskcats="1")
    data = gs.read_command("r.stats", input="vegetated", flags="cnl", separator=",")
    reader = csv.DictReader(
        io.StringIO(data), delimiter=",", fieldnames=["category", "label", "count"]
    )
    result_row = {}
    # Store the viewshed name.
    result_row["viewshed"] = viewshed
    for row in reader:
        # Store count under the corresponding label.
        result_row[row["label"]] = int(row["count"])
    result.append(result_row)
    gs.run_command("r.mask", flags="r")

For further analysis, we convert the table to a pandas DataFrame. 

In [ ]:
import pandas as pd

df = pd.DataFrame(result)

# Seaborn prefers melted DataFrame where rows are measurements,
# category labels are variables, and only one column with counts.
# The resulting DataFrame has only 3 columns, but 2x more rows. 
df_melted = df.melt(id_vars=["viewshed"], var_name="type", value_name="count")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=df_melted,
    x="viewshed",
    y="count",
    hue="type",
    palette=["#61709c", "#9ed528"],
)
plt.xticks(rotation=90)
plt.xlabel("Viewshed")
plt.ylabel("Count")
plt.title("Comparison of Vegetated and Non-Vegetated Areas in Each Viewshed")
plt.legend(title="ExGI-based Vegetation")
plt.show()

Let's say we are interested in places where the traveler sees most green vegetation, so we will filter viewsheds where vegetated area is higher or nearly equal to non-vegetated area. We will set our threshold for nearly equal to 10%.

In [ ]:
threshold = 0.1

# pandas expression for our condition
selected_viewsheds = df[
    (df["vegetated"] >= df["not vegetated"]) |
    (abs(df["vegetated"] - df["not vegetated"]) /
     (df["vegetated"] + df["not vegetated"]) <= threshold
    )
]
selected_viewsheds

With the list of viewsheds, we can look where there are:

In [ ]:
green_map = gj.Map(use_region=True)
green_map.d_rast(map="ortho")
green_map.d_vect(map="umstead_drive", width=3, color="white")
for name in selected_viewsheds["viewshed"]:
    green_map.d_rast(map=name, values=1)
green_map.show()